# Config code

In [1]:
global_config = {
    'tasks':'LTV_Prediction', 
    'data_config':{
        "path": "../dataset/data_online.csv",#"http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx",
        "History":{
            "user_id": "CustomerID",
            "invoice_date": "InvoiceDate",
            "product_id": "StockCode",
            "invoice_no": "InvoiceNo",
            "num_units": "Quantity",
            "unit_price": "UnitPrice",
            "metadata": {}},
        'Constraint': {
            'event_constraint': 0.5 # Number or tr months model will use to train 
        }
}}

In [2]:
def cal_TotalPurchase(df, keys, groups, feature_names, target_name):
	tp_df = df.groupby(keys + groups)['TotalPurchase'].sum().reset_index(name = target_name)
	return tp_df

In [3]:
from propensity_prediction.config.base import HistoryBased_DataConfig

class Purchase_DataConfig(HistoryBased_DataConfig):
	def __init__(self, data_config):
		super().__init__(data_config)

# Run

In [4]:
data_config = Purchase_DataConfig(global_config['data_config'])
data_config

In [5]:
import numpy as np
import pandas as pd
df = pd.read_csv("../dataset/data_online.csv", dtype = "str")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [7]:
from propensity_prediction.model.analytics.purchase_processing import PurchaseFeatures_ByAge
fg = PurchaseFeatures_ByAge(data_config)
df_preprocessed = fg.preprocess(df)
fg.update_cusinfo(df_preprocessed)
df_preprocessed = fg.add_PurchaseFeatures(df_preprocessed)
tracking_data = fg.create_tracking_bytarget(df_preprocessed, cal_TotalPurchase, 'LTV')
tracking_data.head()

,CustomerID,Customer_Age,Customer_NextAge,LTV,Next_LTV
0,12346,0,NaN,77183.60,NaN
1,12347,0,1.0,711.79,475.39
2,12348,0,1.0,892.80,227.44
3,12349,0,NaN,1757.55,NaN
4,12350,0,NaN,334.40,NaN


In [8]:
tracking_data[tracking_data['CustomerID'] == '17841']

,CustomerID,Customer_Age,Customer_NextAge,LTV,Next_LTV
4010,17841,0,1.0,2468.12,1949.98
925,17841,1,2.0,1949.98,1007.20
864,17841,2,3.0,1007.20,1468.40
858,17841,3,4.0,1468.40,1766.32
759,17841,4,5.0,1766.32,2619.48
788,17841,5,6.0,2619.48,3020.16
730,17841,6,7.0,3020.16,4180.86
655,17841,7,8.0,4180.86,3735.88
616,17841,8,9.0,3735.88,2905.19
547,17841,9,10.0,2905.19,4983.57


In [9]:
df_preprocessed

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPurchase,Recency,ActiveDays,Customer_Age
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,373,0,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,373,0,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,373,0,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,373,0,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,373,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,10.20,0,112,3
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60,0,112,3
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60,0,112,3
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60,0,112,3


In [ ]:
profile_byage = fg.create_profile(df_preprocessed)
profile_byage

In [ ]:
profile_byage[profile_byage.CustomerID == '17841']

# Draw

In [19]:
def cal_AAtables(df):
    df['LTV_rate'] = df['Next_LTV'] / df['LTV']
    draw_df = df.groupby(['CustomerID', 'InvoiceDate_Range', 'Customer_Age'])[['LTV', 'LTV_rate']].mean().reset_index()
    draw_df['ActiveDate_Range'] = draw_df['InvoiceDate_Range'] - draw_df['Customer_Age'] 
    draw_df.rename(columns = {'Customer_Age': 'Age', 'ActiveDate_Range': 'Active'}, inplace=True)
    draw_df['Age'] = draw_df['Age']
    ltv_table = pd.pivot_table(draw_df, values = 'LTV', index = ['Active'], columns = ['Age'], aggfunc = np.mean)
    cnt_table = pd.pivot_table(draw_df, values = 'LTV', index = ['Active'], columns = ['Age'], aggfunc = np.count_nonzero)
    ltvrate_table = pd.pivot_table(draw_df, values = 'LTV_rate', index = ['Active'], columns = ['Age'], aggfunc = np.mean)
    return {'LTV': ltv_table, 'Number of customers': cnt_table, 'LTV_rate': ltvrate_table}

In [20]:
tables = list(cal_AAtables(tracking_data).items())
print (tables[0][0])
tables[0][1]

LTV


Age,0,1,2,3,4,5,6,7,8,9,10,11,12
Active,,,,,,,,,,,,,
132,880.530855,NaN,1769.638182,1143.045455,1128.111667,2057.310000,2170.486000,3743.252500,7444.196000,911.738333,3631.792857,5009.820000,1499.544706
133,747.823234,864.515856,808.796134,996.304321,746.146160,988.792121,1037.150210,1115.354470,1062.997481,1415.182353,1520.729603,1195.253015,920.625901
134,799.469139,1066.452034,530.297414,1153.119008,885.947034,760.972313,1144.678136,1329.567217,871.325309,2231.223538,1194.967598,1149.522446,NaN
135,485.209627,485.335253,574.533617,514.369191,552.750917,547.663697,600.791524,817.198450,763.208710,623.785862,999.836389,NaN,NaN
136,510.441716,473.118088,504.910547,438.081636,502.001136,530.567765,591.389435,724.400777,580.822448,402.540200,NaN,NaN,NaN
137,493.398132,458.674845,494.383919,402.635109,461.479178,505.187732,531.761294,589.296949,584.529250,NaN,NaN,NaN,NaN
138,488.611552,363.825616,399.497000,375.121186,414.583833,460.530175,504.290349,375.527200,NaN,NaN,NaN,NaN,NaN
139,617.727181,431.896750,446.195122,511.172985,528.658939,446.709359,4926.962105,NaN,NaN,NaN,NaN,NaN,NaN
140,476.282688,461.561034,387.301860,478.006750,448.593125,681.425909,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
print (tables[1][0])
tables[1][1]

Number of customers


Age,0,1,2,3,4,5,6,7,8,9,10,11,12
Active,,,,,,,,,,,,,
132,234.0,NaN,11.0,11.0,12.0,1.0,5.0,8.0,5.0,6.0,14.0,5.0,17.0
133,739.0,292.0,269.0,287.0,250.0,297.0,286.0,264.0,266.0,306.0,277.0,388.0,161.0
134,511.0,118.0,116.0,121.0,145.0,134.0,118.0,115.0,149.0,130.0,204.0,139.0,NaN
135,456.0,99.0,94.0,136.0,109.0,119.0,105.0,129.0,124.0,145.0,72.0,NaN,NaN
136,437.0,68.0,128.0,110.0,88.0,85.0,124.0,103.0,143.0,50.0,NaN,NaN,NaN
137,326.0,97.0,74.0,92.0,73.0,97.0,85.0,118.0,40.0,NaN,NaN,NaN,NaN
138,310.0,73.0,50.0,59.0,60.0,57.0,86.0,25.0,NaN,NaN,NaN,NaN,NaN
139,259.0,40.0,41.0,67.0,66.0,78.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN
140,205.0,29.0,43.0,40.0,64.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
print (tables[2][0])
tables[2][1]

LTV_rate


Age,0,1,2,3,4,5,6,7,8,9,10,11
Active,,,,,,,,,,,,
132,2.195219,NaN,1.638859,0.727031,1.809869,1.014835,0.779712,0.976708,1.742834,2.387282,1.347888,1.107083
133,1.260700,3.169448,3.181032,1.195641,1.822646,1.135387,1.300393,1.255294,1.613057,1.513424,1.550402,0.823307
134,1.022438,1.400481,1.089202,1.137134,1.649996,1.257524,1.241852,1.425618,1.624991,1.166908,1.298509,0.492142
135,2.405576,5.517755,2.436152,0.972892,1.065597,1.385496,1.498381,1.641972,1.090224,1.094800,0.377865,NaN
136,0.991054,4.913033,1.405158,1.178137,1.742010,1.834050,1.449322,1.043521,0.810180,NaN,NaN,NaN
137,0.989617,1.129838,1.170949,1.567580,1.589079,0.965370,7.748489,0.804114,0.338770,NaN,NaN,NaN
138,0.860952,1.308217,1.316269,1.898752,1.712916,1.439588,0.769708,0.747962,NaN,NaN,NaN,NaN
139,0.809797,1.191995,0.951119,1.178441,1.162159,0.906722,0.231885,NaN,NaN,NaN,NaN,NaN
140,0.801814,1.545080,1.303850,1.103590,0.911941,NaN,NaN,NaN,NaN,NaN,NaN,NaN
